## Data Cleaning
In this notebook we seek to aggregate the data we've scraped and gathered, clean it, and prepare it for use in our modelling (whether primarily statistical or ML-based). Data has been extracted from multiple sources (primarily wikipedia tables), saved in excel flat files. 

In [62]:
# import packages
from fuzzywuzzy import process, fuzz
from itertools import repeat
import pandas as pd
import os

### Load Raw Data
Here we load raw data from excel files in the designated directory.

In [63]:
# Retrieve master list of destinations
destinations = pd.read_excel(
    io="Travel_Destinations.xlsx",
    header=None
)[0].values.tolist()

# Retrieve file names and contruct paths for all raw data files
dir_name = "raw_data"
data_files = os.listdir(dir_name)
data_paths = list()
for file_name in data_files:
    data_paths.append((file_name, os.path.join(dir_name, file_name)))

### Fuzzy matching

The goal here is that for each table in an excel flat file, given an instance in the data which has an associated destination, we want to compare the destination in the data with our master list of destinations and use fuzzy matching to select the most appropriate match. We replace the original destination with the matched destination. Upon completion, each data set will have the same set of destinations listed, making aggregation more accurate and feasible.

Fuzzy matching typically uses the Levenshtein distance (aka the edit distance) or Indel distance as a basis. Both compute the number of operations, or edits, required to transform one string into another. The deletion, insertion, or substitution of a character all result in +1 to the Leneshtein distance (all actions are weighted equally). Related, the Indel distance is similar to the Levenshtein distance except it does not permit substitutions. Effectively, any substitution that would have incurred a cost of +1 is replaced by an insertion and deletion with cost +2, or equivalently, a substitution is given a weight of +2. The normalized distance for Levenshtein is computed as $$\frac{distance}{max(len(string1),\ len(string2)}$$ whereas the normalized distance for Indel is computed as $$\frac{distance}{len(string1)\ +\ len(string2)}$$

The similarity between two strings is $1-distance$. When using a specific ratio in the FuzzyWuzzy package, the function preprocesses the input strings in some way and then computes the scaled Levenshtein similarity. The Token Sort Ratio splits a string into its tokens/words, makes them lowercase and removes punctuations, then sorts the tokens alphabetically and joins them prior to computing the similarity.

In [64]:
fuzzy_dataframes = list()
fuzzy_path = "fuzzy_data"
if not os.path.exists(fuzzy_path):
    os.mkdir(fuzzy_path)

# for every file and table within
table_count = 0
for data_path in data_paths:
    file_name = data_path[0]
    file_path = data_path[1]
    table_count += 1
    
    # open the file
    with open(file_path, "rb") as f:
        # read the file and extract its list of destinations
        unmatched_data = pd.read_excel(f, index_col=0)
        unmatched_destinations = unmatched_data.index.values.tolist()
        
        # perform fuzzy match 
        matched_destinations = list(map(
            process.extractOne,  # function to be applied
            unmatched_destinations,  # applied to 
            repeat(destinations),  # compared against
            repeat(process.default_processor),  # use default processor each time
            repeat(fuzz.token_sort_ratio),  # use token sort ratio instead of default
            repeat(75)  # similarity must be higher than 50% to be considered a match
        ))

        # debugging the fuzzy match logic
        unmatched_count = matched_destinations.count(None)
        print(f"Table {table_count} did not match {unmatched_count} entries")
        if unmatched_count > 0:
            unmatched_indices = [i for i, match in enumerate(matched_destinations) if match == None]
        
        # check if any entries are None indicating no match, if so, set to previous value 
        matched_destinations = [unmatched_destinations[index] if match == None else match[0] for index, match in enumerate(matched_destinations)]

        # debugging the fuzzy match logic
        if unmatched_count > 0:
            for i in range(unmatched_count):
                print(f"---{matched_destinations[unmatched_indices[i]]}")
                destinations.append(matched_destinations[unmatched_indices[i]])  # add legitimate unmatched destinations to main list
        
        # reindex the data to the fuzzy matched destinations
        matched_data = unmatched_data.set_index(pd.Index(matched_destinations))

        # save the file name in the fuzzy folder
        matched_data.to_excel(os.path.join(fuzzy_path, file_name))
        fuzzy_dataframes.append(matched_data)
        

Table 1 did not match 0 entries
Table 2 did not match 0 entries
Table 3 did not match 0 entries
Table 4 did not match 0 entries
Table 5 did not match 0 entries
Table 6 did not match 0 entries
Table 7 did not match 0 entries
Table 8 did not match 0 entries
Table 9 did not match 0 entries
Table 10 did not match 0 entries
Table 11 did not match 0 entries
Table 12 did not match 0 entries
Table 13 did not match 0 entries
Table 14 did not match 0 entries
Table 15 did not match 0 entries
Table 16 did not match 0 entries
Table 17 did not match 0 entries
Table 18 did not match 0 entries
Table 19 did not match 0 entries
Table 20 did not match 0 entries
Table 21 did not match 0 entries
Table 22 did not match 0 entries
Table 23 did not match 0 entries
Table 24 did not match 0 entries
Table 25 did not match 0 entries
Table 26 did not match 0 entries


In [65]:
# outer join all data based on the destinations
joined_data = fuzzy_dataframes[2].join(
    fuzzy_dataframes[0:2]+fuzzy_dataframes[3:],
    how="outer",
    sort=True
)

# remove duplicate rows that end up in the data
joined_data = joined_data[~joined_data.index.duplicated(keep='first')]

### Imputation

Here we perform imputation to fill in missing values. There are no destinations in this data that have all missing values. We do not drop any destinations from the analysis as we prefer to have a broader set to recommend to end-users.

In this analysis we use multiple imputation by chained equations (MICE) as our imputation method, due to its attempts to account for the uncertainty in the imputed value by provingd both within-imputation and between-imputation variability, a characteristic which makes multiple imputation preferable over single imputation in most circumstances.

In [ ]:


len(joined_data[joined_data.count(axis=1) <= joined_data.shape[1]*.35])

In [95]:
joined_data.shape[0]

237

64